<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>
<font size="4"><center> <b> Example 2 - Sand labelling </b></center> <br>

    
<center><img src="images/banner.png" width="80%"  /></center>

<font face="Calibri">
<br>
<font size="5"> <b>Sand clustering with Silhouette Analysis and KMeans notebook</b></font>

<br>
<font size="4"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<font size="3">This notebook illustrates how to use Sandpyper to perform Silhouette Analysis and KMeans on all previously extracted points. <br>

<b>This notebook covers the following concepts:</b>

- Silhouete Analysis.
- KMeans clustering.
</font>


</font>

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from sandpyper.outils import coords_to_points 
from sandpyper.labels import get_sil_location, get_opt_k, kmeans_sa

pd.options.mode.chained_assignment = None  # default='warn'

C:\conda3\envs\sandpyper_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Loading the project-related lists

- loc codes
- crs dict string

In [2]:
# The location codes used troughout the analysis
loc_codes=["mar","leo"]

# The Coordinate Reference Systems used troughout this study
crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo': {'init': 'epsg:32755'},
                 }

## Loading, merging and preparing the tables

The function __get_merged_table__ merge the rgb and z tables together and format it in a way it is digestible for further analysis.

In [3]:
%%time

#Loading the tables

rgb_table_path=r"C:\my_packages\sandpyper\tests\test_outputs\gdf_rgb.csv"
z_table_path=r"C:\my_packages\sandpyper\tests\test_outputs\gdf.csv"

rgb_table=gpd.read_file(rgb_table_path)
z_table=gpd.read_file(z_table_path)

# As the distance (across-transect) comes from an interpolation, it has too many digits.
# let's round both tables distance columns to 2 significant values and assign their data type as "float".

rgb_table["distance"]=np.round(rgb_table.loc[:,"distance"].values.astype("float"),2)
z_table["distance"]=np.round(z_table.loc[:,"distance"].values.astype("float"),2)

C:\conda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Wall time: 4.67 s


Storing Geodataframes as CSV is handy, but __we lose the column data type information__.
Especially important is the __geometry column__, which we need to convert back into __Shapely Point object format__.
To do that, the function __coords_to_points__ can be used across a Series ('geometry'). It can take quite a bit of time, so, if you have a lot of points, get ready!

In [5]:
# Here, we merge the two tables (storing elevation and rgb information)

data_merged = pd.merge(z_table,rgb_table[["band1","band2","band3","point_id"]],on="point_id",validate="one_to_one")

# replace empty values with np.NaN
data_merged=data_merged.replace("", np.NaN)

# and convert the z column into floats.
data_merged['z']=data_merged.z.astype("float")

data_merged.head()

,distance,z,tr_id,raw_date,coordinates,location,survey_date,point_id,x,y,geometry,band1,band2,band3
0,0.0,0.007440,21,20190516,POINT (731646.903760184 5705523.468988597),mar,2019-05-16,61121091m2580400ar00,731646.903760184,5705523.468988597,POINT (731646.904 5705523.469),114.0,139.0,128.0
1,1.0,0.008439,21,20190516,POINT (731646.0783010386 5705524.033450465),mar,2019-05-16,61123091m2580600ar10,731646.0783010386,5705524.033450465,POINT (731646.078 5705524.033),117.0,139.0,127.0
2,2.0,0.010800,21,20190516,POINT (731645.2528418931 5705524.597912331),mar,2019-05-16,61129091m2530100ar20,731645.2528418931,5705524.597912331,POINT (731645.253 5705524.598),122.0,140.0,127.0
3,3.0,0.011350,21,20190516,POINT (731644.4273827478 5705525.162374198),mar,2019-05-16,61124091m2570800ar30,731644.4273827478,5705525.162374198,POINT (731644.427 5705525.162),125.0,144.0,133.0
4,4.0,0.028030,21,20190516,POINT (731643.6019236024 5705525.726836066),mar,2019-05-16,61120091m2520400ar40,731643.6019236024,5705525.726836066,POINT (731643.602 5705525.727),126.0,145.0,133.0


In [ ]:
# Here, we add two features, slope and curvature, computed from the elevation series,
# in case we wnat to use for KMeans clustering.
# Note that when passing from one transect to another, slope and curvature computations are wrong.
# However, we will clip those areas as they are in the water or in the backdune.

data_merged["slope"]=np.gradient(data_merged.z)
data_merged["curve"]=np.gradient(data_merged.slope)

data_merged.head()

In [6]:
# Our rasters have NaN values set to -32767.0. Thus, we replace them with np.Nan.
data_merged.z.replace(-32767.0,np.nan,inplace=True)

## Iterative silhouette analysis


The __get_sil_location__ function will iteratively perform KMeans clustering and Silhouette Analysis with increasing number of clusters (k, specified in the `ks` parameter) for every survey, using the feature set specified in the parameter `feature_set`.

This will return a dataframe with Average Silhouette scores with different k for all surveys, which we use to find sub-optimal number of clusters with __get_opt_k__ function.

Then, with the sub-optimal k, we finally run KMeans with __kmeans_sa__ function on all the surveys to obtain clustered points to visually discriminate between sand and non-sand in a Qgis environment.

In [7]:
%%time
# Run interatively KMeans + SA

feature_set=["band1","band2","band3","distance"]
sil_df=get_sil_location(data_merged,
                        ks=(2,15), 
                        feature_set=feature_set,
                       random_state=10)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Working on : mar, 20190516.


  0%|          | 0/13 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.6219865168763407
For n_clusters = 3 The average silhouette_score is : 0.535782834177223
For n_clusters = 4 The average silhouette_score is : 0.5360693626824162
For n_clusters = 5 The average silhouette_score is : 0.4560535847617118
For n_clusters = 6 The average silhouette_score is : 0.45529082978244856
For n_clusters = 7 The average silhouette_score is : 0.44623104595368007
For n_clusters = 8 The average silhouette_score is : 0.4276467073078296
For n_clusters = 9 The average silhouette_score is : 0.39984472497160034
For n_clusters = 10 The average silhouette_score is : 0.3944833658664287
For n_clusters = 11 The average silhouette_score is : 0.39290037915600995
For n_clusters = 12 The average silhouette_score is : 0.3626866522843635
For n_clusters = 13 The average silhouette_score is : 0.3537214969510808
For n_clusters = 14 The average silhouette_score is : 0.360049558138258
Working on : mar, 20190313.


...

For n_clusters = 2 The average silhouette_score is : 0.4039166622963731
For n_clusters = 3 The average silhouette_score is : 0.4367493135688848
For n_clusters = 4 The average silhouette_score is : 0.42434419318841404
For n_clusters = 5 The average silhouette_score is : 0.4443994359362926
For n_clusters = 6 The average silhouette_score is : 0.4175864662989779
For n_clusters = 7 The average silhouette_score is : 0.38860870508958795
For n_clusters = 8 The average silhouette_score is : 0.38880003508342514
For n_clusters = 9 The average silhouette_score is : 0.3830287983992327
For n_clusters = 10 The average silhouette_score is : 0.37353163965128733
For n_clusters = 11 The average silhouette_score is : 0.36620965721819637
For n_clusters = 12 The average silhouette_score is : 0.3536600836899299
For n_clusters = 13 The average silhouette_score is : 0.3585083805642395
For n_clusters = 14 The average silhouette_score is : 0.35672016047028215
Working on : leo, 20180713.


  0%|          | 0/13 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.49629965872244686
For n_clusters = 3 The average silhouette_score is : 0.5092406492417487
For n_clusters = 4 The average silhouette_score is : 0.46159478658042724
For n_clusters = 5 The average silhouette_score is : 0.4488595142176017
For n_clusters = 6 The average silhouette_score is : 0.4183623304561938
For n_clusters = 7 The average silhouette_score is : 0.39004221840210485
For n_clusters = 8 The average silhouette_score is : 0.3842278112748862
For n_clusters = 9 The average silhouette_score is : 0.36304035595403583
For n_clusters = 10 The average silhouette_score is : 0.3463760386521644
For n_clusters = 11 The average silhouette_score is : 0.34841430296451464
For n_clusters = 12 The average silhouette_score is : 0.3460546307634773
For n_clusters = 13 The average silhouette_score is : 0.34457917043266856
For n_clusters = 14 The average silhouette_score is : 0.3318667826222712
Working on : leo, 20180606.


  0%|          | 0/13 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.4012079287817844
For n_clusters = 3 The average silhouette_score is : 0.4563078554750914
For n_clusters = 4 The average silhouette_score is : 0.40640166442880776
For n_clusters = 5 The average silhouette_score is : 0.4001757072050638
For n_clusters = 6 The average silhouette_score is : 0.3890044660503746
For n_clusters = 7 The average silhouette_score is : 0.3716889344520391
For n_clusters = 8 The average silhouette_score is : 0.36600629265911533
For n_clusters = 9 The average silhouette_score is : 0.3503569988098764
For n_clusters = 10 The average silhouette_score is : 0.34651080656561367
For n_clusters = 11 The average silhouette_score is : 0.34322073624826754
For n_clusters = 12 The average silhouette_score is : 0.3439561998875081
For n_clusters = 13 The average silhouette_score is : 0.347961415981389
For n_clusters = 14 The average silhouette_score is : 0.33782737875946023
Wall time: 47.5 s


In [8]:
sil_df.head()

,location,raw_date,k,silhouette_mean
0,mar,20190516,2,0.621987
1,mar,20190516,3,0.535783
2,mar,20190516,4,0.536069
3,mar,20190516,5,0.456054
4,mar,20190516,6,0.455291


##  Sub-optimal k

Find sub-optimal k by searching inflexion points where an additional cluster do not considerably degrade the overall clustering performance.

In [9]:
opt_k=get_opt_k(sil_df, sigma=0 )
opt_k

{'leo_20180606': 11,
 'leo_20180713': 10,
 'leo_20180920': 4,
 'leo_20190211': 4,
 'leo_20190328': 7,
 'leo_20190731': 9,
 'mar_20180601': 3,
 'mar_20180621': 3,
 'mar_20180727': 11,
 'mar_20180925': 7,
 'mar_20181113': 5,
 'mar_20181211': 5,
 'mar_20190205': 6,
 'mar_20190313': 6,
 'mar_20190516': 3}

If we are not satisfied with the sub-optimal k returned by the algorithm, we can manually specify each survey k
by defining a dictionary.

In [ ]:
# Based on our observations on a dataset comprising 87 surveys, 10 clusters (k=10) is generally a good tradeoff.

opt_k={'leo_2018-06-06': 10,
 'leo_2018-07-13': 10,
 'leo_2018-09-20': 10,
 'leo_2019-02-11': 10,
 'leo_2019-03-28': 10,
 'leo_2019-07-31': 10,
 'mar_2018-06-01': 10,
 'mar_2018-06-21': 10,
 'mar_2018-07-27': 10,
 'mar_2018-09-25': 10,
 'mar_2018-11-13': 10,
 'mar_2018-12-11': 10,
 'mar_2019-02-05': 10,
 'mar_2019-03-13': 10,
 'mar_2019-05-16': 10}

opt_k

or, update one value only. For instance, in mar_2019-05-16 dataset, it is unlikely that 3 clusters are enough.<br>
So, we replace only that value with 10.


In [ ]:
opt_k['mar_2019-05-16']=10
opt_k

## Optimised K-Means clustering

With the sub-optimal k dictionary and keeping the same feature set, we finally cluster the dataset.

In [10]:
data_merged.columns

Index(['distance', 'z', 'tr_id', 'raw_date', 'coordinates', 'location',
       'survey_date', 'point_id', 'x', 'y', 'geometry', 'band1', 'band2',
       'band3'],
      dtype='object')

In [12]:
data_classified.columns

Index(['distance', 'z', 'tr_id', 'raw_date', 'coordinates', 'location',
       'survey_date', 'point_id', 'x', 'y', 'geometry', 'band1', 'band2',
       'band3', 'label_k'],
      dtype='object')

In [11]:
feature_set=["band1","band2","band3","distance"]
data_classified=kmeans_sa(data_merged,opt_k, feature_set=feature_set)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
data_classified2 = pd.merge(data_classified[["point_id","label_k"]], data_merged, how="left", on="point_id", validate="one_to_one")

In [18]:
from sandpyper.dynamics import compute_multitemporal
from sandpyper.outils import create_details_df, create_spatial_id
loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}

In [21]:
def compute_multitemporal (df,
                           filter_sand=True,
                           date_field='survey_date',
                          sand_label_field='label_sand',
                           filter_classes=[0]):
    """
    From a dataframe containing the extracted points and a column specifying wether they are sand or non-sand, returns a multitemporal dataframe
    with time-periods sand-specific elevation changes.

    Args:
        date_field (str): the name of the column storing the survey date.
        sand_label_field (str): the name of the column storing the sand label (usually sand=0, no_sand=1).
        filter_classes (list): list of integers specifiying the label numbers of the sand_label_field that are sand. Default [0].
        common_field (str): name of the field where the points share the same name. It is usually the geometry or spatial IDs.

    Returns:
        A multitemporal dataframe of sand-specific elevation changes.
    """

    df["spatial_id"]=[create_spatial_id(df.iloc[i]) for i in range(df.shape[0])]
    fusion_long=pd.DataFrame()

    for location in df.location.unique():
        print(f"working on {location}")
        loc_data=df.query(f"location=='{location}'")
        list_dates=loc_data.loc[:,date_field].unique()
        list_dates.sort()


        for i in range(list_dates.shape[0]):

            if i < list_dates.shape[0]-1:
                date_pre=list_dates[i]
                date_post=list_dates[i+1]
                print(f"Calculating dt{i}, from {date_pre} to {date_post} in {location}.")

                if filter_sand:
                    df_pre=loc_data.query(f"{date_field} =='{date_pre}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
                    df_post=loc_data.query(f"{date_field} =='{date_post}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
                else:
                    df_pre=loc_data.query(f"{date_field} =='{date_pre}'").dropna(subset=['z'])
                    df_post=loc_data.query(f"{date_field} =='{date_post}'").dropna(subset=['z'])

                merged=pd.merge(df_pre,df_post, how='inner', on='spatial_id', validate="one_to_one",suffixes=('_pre','_post'))
                merged["dh"]=merged.z_post.astype(float) - merged.z_pre.astype(float)

                dict_short={"geometry": merged.geometry_pre,
                            "location":location,
                            "tr_id":merged.tr_id_pre,
                            "distance":merged.distance_pre,
                            "dt":  f"dt_{i}",
                            "date_pre":date_pre,
                            "date_post":date_post,
                            "z_pre":merged.z_pre.astype(float),
                            "z_post":merged.z_post.astype(float),
                            "dh":merged.dh}

                short_df=pd.DataFrame(dict_short)
                fusion_long=pd.concat([short_df,fusion_long],ignore_index=True)

    print("done")
    return fusion_long

In [24]:
dh_df=compute_multitemporal(data_classified2,
                     date_field='raw_date', filter_sand=False,
                     sand_label_field='label_sand')

working on leo
Calculating dt0, from 20180606 to 20180713 in leo.
Calculating dt1, from 20180713 to 20180920 in leo.
Calculating dt2, from 20180920 to 20190211 in leo.
Calculating dt3, from 20190211 to 20190328 in leo.
Calculating dt4, from 20190328 to 20190731 in leo.
working on mar
Calculating dt0, from 20180601 to 20180621 in mar.
Calculating dt1, from 20180621 to 20180727 in mar.
Calculating dt2, from 20180727 to 20180925 in mar.
Calculating dt3, from 20180925 to 20181113 in mar.
Calculating dt4, from 20181113 to 20181211 in mar.
Calculating dt5, from 20181211 to 20190205 in mar.
Calculating dt6, from 20190205 to 20190313 in mar.
Calculating dt7, from 20190313 to 20190516 in mar.
done


In [41]:
dh_df

,geometry,location,tr_id,distance,dt,date_pre,date_post,z_pre,z_post,dh
0,POINT (731646.904 5705523.469),mar,21,0.0,dt_7,20190313,20190516,1.111801,0.007440,-1.104360
1,POINT (731646.078 5705524.033),mar,21,1.0,dt_7,20190313,20190516,1.124138,0.008439,-1.115699
2,POINT (731645.253 5705524.598),mar,21,2.0,dt_7,20190313,20190516,1.117822,0.010800,-1.107022
3,POINT (731644.427 5705525.162),mar,21,3.0,dt_7,20190313,20190516,1.148563,0.011350,-1.137213
4,POINT (731643.602 5705525.727),mar,21,4.0,dt_7,20190313,20190516,1.112438,0.028030,-1.084408
...,...,...,...,...,...,...,...,...,...,...
19862,POINT (300071.060 5773184.013),leo,18,48.0,dt_0,20180606,20180713,-0.435838,-0.624926,-0.189088
19863,POINT (300072.023 5773184.284),leo,18,49.0,dt_0,20180606,20180713,-0.431080,-0.642019,-0.210939
19864,POINT (300075.506 5773164.488),leo,17,47.0,dt_0,20180606,20180713,-0.316839,-0.478775,-0.161936
19865,POINT (300076.469 5773164.759),leo,17,48.0,dt_0,20180606,20180713,-0.459795,-0.477090,-0.017296


In [15]:
dh_df.to_csv(r"C:\my_packages\sandpyper\tests\test_outputs\dh_data.csv", index=False)

In [42]:
details=create_details_df(dh_df, loc_full)

In [43]:
details

,dt,date_pre,date_post,location,n_days,loc_full
0,dt_0,20180606,20180713,leo,37,St. Leonards
1,dt_1,20180713,20180920,leo,69,St. Leonards
2,dt_2,20180920,20190211,leo,144,St. Leonards
3,dt_3,20190211,20190328,leo,45,St. Leonards
4,dt_4,20190328,20190731,leo,125,St. Leonards
5,dt_0,20180601,20180621,mar,20,Marengo
6,dt_1,20180621,20180727,mar,36,Marengo
7,dt_2,20180727,20180925,mar,60,Marengo
8,dt_3,20180925,20181113,mar,49,Marengo
9,dt_4,20181113,20181211,mar,28,Marengo


In [22]:
details.to_csv(r"C:\my_packages\sandpyper\tests\test_outputs\dt_info.csv", index=False)

### GOOD!

save the __data_classified__ dataframe as a CSV file and head to the __Example_3_Labels_correction_and_multitemporal_table notebook__.

In [ ]:
data_classified.to_csv(r"C:\my_packages\sandpyper\tests\test_outputs\data_classified.csv", index=False)

___